In [2]:
import pandas as pd

In [10]:
folderlocation = "D:/PycharmProjects/TerraBot/terra-mystica"
gamesraw = pd.read_csv(f'{folderlocation}/games.csv')
gamesraw.head()

,base_map,game,last_update,player_count
0,126fe960806d587c78546b30f1a90853b1ada468,0000000001,2015-07-22 05:15:51,2
1,be8f6ebf549404d015547152d5f2a1906ae8dd90,050615,2015-07-13 09:45:00,4
2,126fe960806d587c78546b30f1a90853b1ada468,0512,2015-07-03 04:17:31,4
3,126fe960806d587c78546b30f1a90853b1ada468,051501,2015-07-16 21:00:54,5
4,95a66999127893f5925a5f591d54f8bcb9a670e6,060303,2015-07-02 05:20:19,4


In [11]:
gamesraw.base_map.value_counts()

126fe960806d587c78546b30f1a90853b1ada468    77158
95a66999127893f5925a5f591d54f8bcb9a670e6    21052
be8f6ebf549404d015547152d5f2a1906ae8dd90    16184
fdb13a13cd48b7a3c3525f27e4628ff6905aa5b1     1210
224736500d20520f195970eb0fd4c41df040c08c        3
735b073fd7161268bb2796c1275abda92acd8b1a        2
c07f36f9e050992d2daf6d44af2bc51dca719c46        1
30b6ded823e53670624981abdb2c5b8568a44091        1
b109f78907d2cbd5699ced16572be46043558e41        1
Name: base_map, dtype: int64

In [7]:
def select_games():
    """We want games that:
    1. Only use the original map
    2. Only have 4 players
    3. Have original tiles selected ...?
    """


    folderlocation = "D:/PycharmProjects/TerraBot/terra-mystica"

    stats = pd.read_csv(f'{folderlocation}/games.csv')

    """
    input: 
    >> stats.base_map.value_counts(ii)
    Output:
    126fe960806d587c78546b30f1a90853b1ada468    0.667387  << this guy's our guy
    95a66999127893f5925a5f591d54f8bcb9a670e6    0.182092
    be8f6ebf549404d015547152d5f2a1906ae8dd90    0.139985
    fdb13a13cd48b7a3c3525f27e4628ff6905aa5b1    0.010466
    224736500d20520f195970eb0fd4c41df040c08c    0.000026
    735b073fd7161268bb2796c1275abda92acd8b1a    0.000017
    b109f78907d2cbd5699ced16572be46043558e41    0.000009
    c07f36f9e050992d2daf6d44af2bc51dca719c46    0.000009
    30b6ded823e53670624981abdb2c5b8568a44091    0.000009
    Name: base_map, dtype: float64"""

    # only use games that have the original map
    processedstats = stats[stats['base_map'] == '126fe960806d587c78546b30f1a90853b1ada468']

    # only use games that have 4 players
    processedstats = processedstats[processedstats['player_count'] == 4]

    # remove nas
    selectedgames = processedstats[~processedstats.isnull().any(axis=1)]

    return selectedgames


So it turns out:
91645cdb135773c2a7a50e5ca9cb18af54c664c4 is the same map as 126f... only there are the following starting vp for each faction:
fakirs	33  
alchemists	27  
auren	27  
giants	25  
dragonlords	24  
acolytes	23  
swarmlings	22  
shapeshifters	22  
yetis	22  
riverwalkers	21  
dwarves	20  
icemaidens	20  
halflings	20  
chaosmagicians	19  
mermaids	19  
nomads	19  
witches	19  
engineers	16  
cultists	16  
darklings	15  

With 126f..., all the factions just start with the base 20. We can use both maps if we deducted/added the point difference from 916, or vice versa. That way we can use this picker for both map layouts.

In [9]:
selectedgames = select_games()
len(selectedgames)

48737

In [12]:
gamesscoretiles = pd.read_csv(f'{folderlocation}/game_scoring_tiles.csv')
gamesscoretiles.head()

,game,round,tile
0,0000000001,3,SCORE1
1,0000000001,6,SCORE5
2,0000000001,5,SCORE4
3,0000000001,1,SCORE8
4,0000000001,2,SCORE3


What does SCORE1, SCORE5, etc relate to? Which scores? And are there added scores that don't come from the vanilla set? Do we want to keep those?

In [16]:
# trying to find the game so that we can look at the round tiles used & correlated to what's on snellman is posing issues
selectedgames[selectedgames['game'] == 'jsnell19']

,base_map,game,last_update,player_count
50412,126fe960806d587c78546b30f1a90853b1ada468,jsnell19,2013-10-30 22:01:28,3


In [18]:
# ah we must have filtered these ones out (need to look into why) (EDIT: it's because it was a 3 player game. duh.)
gamesraw[gamesraw['game'] == 'jsnell19']

,base_map,game,last_update,player_count
50412,126fe960806d587c78546b30f1a90853b1ada468,jsnell19,2013-10-30 22:01:28,3


In [19]:
gamesscoretiles[gamesscoretiles['game'] == 'jsnell19']

,game,round,tile
302472,jsnell19,1,SCORE6
302473,jsnell19,2,SCORE5
302474,jsnell19,3,SCORE4
302475,jsnell19,5,SCORE7
302476,jsnell19,6,SCORE3
302477,jsnell19,4,SCORE2


Great! So that works. Let's have a look...  
SCORE2 = TOWN >> 5  
SCORE3 = D >> 2  
SCORE4 = SA/SH >> 5  
SCORE5 = D >> 2  
SCORE6 = TP >> 3  
SCORE7 = SA/SH >> 5

In [21]:
# hm. How many scores do we have to do?
pd.unique(gamesscoretiles['tile'])

array(['SCORE1', 'SCORE5', 'SCORE4', 'SCORE8', 'SCORE3', 'SCORE2',
       'SCORE6', 'SCORE7', 'SCORE9'], dtype=object)

In [22]:
gamesscoretiles[gamesscoretiles['game'] == 'CatGeorgeGuyJames']

,game,round,tile
243744,CatGeorgeGuyJames,5,SCORE6
243745,CatGeorgeGuyJames,4,SCORE9
243746,CatGeorgeGuyJames,3,SCORE4
243747,CatGeorgeGuyJames,2,SCORE7
243748,CatGeorgeGuyJames,1,SCORE8
243749,CatGeorgeGuyJames,6,SCORE2


In [24]:
gamesscoretiles[gamesscoretiles['game'] == 'GuyJamesChari']

,game,round,tile
484152,GuyJamesChari,1,SCORE1
484153,GuyJamesChari,6,SCORE9
484154,GuyJamesChari,2,SCORE5
484155,GuyJamesChari,3,SCORE4
484156,GuyJamesChari,5,SCORE7
484157,GuyJamesChari,4,SCORE8


So to get all tiles:  
SCORE1 = SPADE >> 2  
SCORE2 = TOWN >> 5  
SCORE3 = D >> 2  
SCORE4 = SA/SH >> 5  
SCORE5 = D >> 2  
SCORE6 = TP >> 3  
SCORE7 = SA/SH >> 5  
SCORE8 = TP >> 3  
SCORE9 = TE >> 4

COOL. So now we've got the map figured out, we've got the scoring tiles, what about the bonus tiles?

In [27]:
# this one?
game_options = pd.read_csv(f'{folderlocation}/game_options.csv')
game_options.head(10)

,game,option
0,0000000001,option-strict-darkling-sh
1,0000000001,option-email-notify
2,0000000001,option-shipping-bonus
3,0000000001,option-strict-chaosmagician-sh
4,0000000001,option-errata-cultist-power
5,0000000001,option-mini-expansion-1
6,0000000001,option-strict-leech
7,050615,option-email-notify
8,050615,option-strict-chaosmagician-sh
9,050615,option-errata-cultist-power


In [28]:
# nope. This one?
game_eventsraw = pd.read_csv(f'{folderlocation}/game_events.csv')

In [33]:
# game_eventsraw.query('game == GuyJamesChari & round == 1')
singlegame = game_eventsraw[(game_eventsraw['game'] == 'GuyJamesChari')]

In [34]:
print(pd.unique(singlegame['round']))

[2 6 1 5 3 4 0]


In [35]:
# ah so there's a round 0
singlegame[singlegame['round'] == 0]

,event,faction,game,num,round,turn
51548368,pass:BON1,nomads,GuyJamesChari,1,0,0
51548421,build:D,nomads,GuyJamesChari,3,0,0
51548516,pass:BON8,engineers,GuyJamesChari,1,0,0
51548560,build:D,engineers,GuyJamesChari,2,0,0
51548663,pass:BON5,cultists,GuyJamesChari,1,0,0
51548709,build:D,cultists,GuyJamesChari,2,0,0
51548824,pass:BON1,all,GuyJamesChari,1,0,0
51548904,pass:BON5,all,GuyJamesChari,1,0,0
51548908,pass:BON8,all,GuyJamesChari,1,0,0
51548996,build:D,all,GuyJamesChari,7,0,0


In [45]:
uniqueroundzero = pd.unique(game_eventsraw[game_eventsraw['round'] == 0]['event'])
print(sorted(uniqueroundzero))

['build:D', 'burn', 'drop', 'favor:FAV1', 'favor:FAV10', 'favor:FAV11', 'favor:FAV12', 'favor:FAV2', 'favor:FAV3', 'favor:FAV4', 'favor:FAV5', 'favor:FAV6', 'favor:FAV7', 'favor:FAV8', 'favor:FAV9', 'favor:any', 'pass:BON1', 'pass:BON10', 'pass:BON2', 'pass:BON3', 'pass:BON4', 'pass:BON5', 'pass:BON6', 'pass:BON7', 'pass:BON8', 'pass:BON9', 'pick-color:black', 'pick-color:blue', 'pick-color:brown', 'pick-color:gray', 'pick-color:green', 'pick-color:red', 'pick-color:yellow', 'vp']


So we have:  
pass:BON1  
pass:BON2  
pass:BON3  
pass:BON4  
pass:BON5  
pass:BON6  
pass:BON7  
pass:BON8  
pass:BON9  
pass:BON10